In [1]:
from pathlib import Path
import re
import sys
from urllib.request import urlretrieve
import xml.etree.ElementTree as ET
import zipfile

PROJECT_ROOT_DIR = Path("..").resolve()
DATA_DIR = PROJECT_ROOT_DIR.joinpath("data")
DATA_DIR.mkdir(exist_ok=True)
SRC_DIR = PROJECT_ROOT_DIR.joinpath("src")
SRC_DIR.mkdir(exist_ok=True)

sys.path.append(str(SRC_DIR))

# Setting up embeddings

In [2]:
url = "https://raw.githubusercontent.com/rasbt/LLMs-from-scratch/main/ch02/01_main-chapter-code/the-verdict.txt"
file_path = DATA_DIR.joinpath("ch02_the_verdict.txt")
if not file_path.is_file():
    urlretrieve(url, file_path)

In [3]:
with open(file_path, "r", encoding="utf-8") as f:
    raw_text = f.read()
print("Total number of character:", len(raw_text))
print(raw_text[:99])

Total number of character: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


## Tokenizing

In [4]:
# Splitting between whitespace and words
text = "Hello, world. This, is a test."
result = re.split(r'(\s)', text)
print(result)

['Hello,', ' ', 'world.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test.']


In [5]:
# Splitting on periods or commas or on whitespace
result = re.split(r'([,.]|\s)', text)
print(result)

['Hello', ',', '', ' ', 'world', '.', '', ' ', 'This', ',', '', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']


In [6]:
# Adding in another stripping op
result = [item for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'This', ',', 'is', 'a', 'test', '.']


In [7]:
# Adding in di
text = "Hello, world. Is this-- a test?"
result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
result = [item.strip() for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


In [8]:
# Using our tokenization pattern on the main text
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(f"Tokens: {len(preprocessed)}")
print(preprocessed[:30])

Tokens: 4690
['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


## Encoding tokens to token_ids

In [9]:
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)
print(vocab_size)

1130


In [10]:
vocab = {token: integer for integer, token in enumerate(all_words)}
for token, i in vocab.items():
    print(f"token: {token:>10}, i: {i}")
    if i >= 50:
        break

token:          !, i: 0
token:          ", i: 1
token:          ', i: 2
token:          (, i: 3
token:          ), i: 4
token:          ,, i: 5
token:         --, i: 6
token:          ., i: 7
token:          :, i: 8
token:          ;, i: 9
token:          ?, i: 10
token:          A, i: 11
token:         Ah, i: 12
token:      Among, i: 13
token:        And, i: 14
token:        Are, i: 15
token:       Arrt, i: 16
token:         As, i: 17
token:         At, i: 18
token:         Be, i: 19
token:      Begin, i: 20
token: Burlington, i: 21
token:        But, i: 22
token:         By, i: 23
token:      Carlo, i: 24
token:    Chicago, i: 25
token:     Claude, i: 26
token:       Come, i: 27
token:      Croft, i: 28
token:  Destroyed, i: 29
token: Devonshire, i: 30
token:        Don, i: 31
token:    Dubarry, i: 32
token:   Emperors, i: 33
token:   Florence, i: 34
token:        For, i: 35
token:    Gallery, i: 36
token:     Gideon, i: 37
token:    Gisburn, i: 38
token:   Gisburns, i: 39
token:    

### Tokenizer v1

In [11]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab            #1
        self.int_to_str = {i:s for s,i in vocab.items()}        #2

    def encode(self, text):         #3
        preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):         #4
        text = " ".join([self.int_to_str[i] for i in ids])

        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)    #5
        return text

tokenizer = SimpleTokenizerV1(vocab)
text = """"It's the last he painted, you know,"
       Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print(ids)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


In [12]:
# Now to decode token_ids back to tokens
tokenizer.decode(ids)

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

In [13]:
# Attempting to use our tokenizer with text containing tokens not in vocab
text = "Hello, do you like tea?"
print(tokenizer.encode(text))

KeyError: 'Hello'

### Tokenizer V2, adding in context-handling tokens

In [14]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])
vocab = {token: integer for integer, token in enumerate(all_tokens)}

print(len(vocab.items()))

1132


That has 2 more elements than in the V1 vocab.

In [15]:
for item in list(vocab.items())[-5:]:
    print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


In [16]:
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = { i:s for s,i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        preprocessed = [item if item in self.str_to_int            #1
                        else "<|unk|>" for item in preprocessed]

        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])

        text = re.sub(r'\s+([,.:;?!"()\'])', r'\1', text)    #2
        return text

In [17]:
text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."
text = " <|endoftext|> ".join((text1, text2))
print(text)
tokenizer = SimpleTokenizerV2(vocab)
tokenizer.encode(text)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]

In [18]:
vocab["<|unk|>"]

1131

In [19]:
tokenizer.decode(tokenizer.encode(text))

'<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.'

### Byte pair encoding

In [20]:
import tiktoken

tiktoken.__version__

'0.9.0'

In [21]:
tokenizer = tiktoken.get_encoding("gpt2")

text = "Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace."
integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
integers

[15496,
 11,
 466,
 345,
 588,
 8887,
 30,
 220,
 50256,
 554,
 262,
 4252,
 18250,
 8812,
 2114,
 1659,
 617,
 34680,
 27271,
 13]

In [22]:
strings = tokenizer.decode(integers)
strings

'Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.'

# Data sampling with a sliding (context) window

In [23]:
enc_text = tokenizer.encode(raw_text)
print(f"Byte encoded tokens: {len(enc_text)}")

Byte encoded tokens: 5145


In [24]:
enc_sample = enc_text[50:]

context_size = 4
x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]
print(f"x: {x}")
print(f"y:      {y}")

x: [290, 4920, 2241, 287]
y:      [4920, 2241, 287, 257]


In [25]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(context, "---->", desired)

[290] ----> 4920
[290, 4920] ----> 2241
[290, 4920, 2241] ----> 287
[290, 4920, 2241, 287] ----> 257


In [26]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(f"{tokenizer.decode(context)} ----> {tokenizer.decode([desired])}")

 and ---->  established
 and established ---->  himself
 and established himself ---->  in
 and established himself in ---->  a


In [27]:
import torch
from torch.utils.data import Dataset, DataLoader

In [28]:
class GPTDatasetV1(Dataset):
    def __init__(
        self,
        txt: str,
        tokenizer: tiktoken.core.Encoding,
        max_length: int,
        stride: int,
    ):
        self.input_ids = []
        self.target_ids = []

        token_ids = tokenizer.encode(txt)

        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]


def create_dataloader_v1(
    txt: str,
    batch_size: int = 4,
    max_length: int = 256,
    stride: int = 128,
    shuffle: bool = True,
    drop_last: bool = True,
    num_workers: int = 0,
) -> DataLoader:
    tokenizer = tiktoken.get_encoding("gpt2")
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers,
    )

    return dataloader

In [29]:
dataloader = create_dataloader_v1(
    raw_text, batch_size=1, max_length=4, stride=1, shuffle=False)
data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


In [30]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


In [31]:
print(next(data_iter))
print(next(data_iter))
print(next(data_iter))
print(next(data_iter))

[tensor([[2885, 1464, 1807, 3619]]), tensor([[1464, 1807, 3619,  402]])]
[tensor([[1464, 1807, 3619,  402]]), tensor([[1807, 3619,  402,  271]])]
[tensor([[1807, 3619,  402,  271]]), tensor([[ 3619,   402,   271, 10899]])]
[tensor([[ 3619,   402,   271, 10899]]), tensor([[  402,   271, 10899,  2138]])]


With a stride of 4 and a max_length of 4, we prevent having given tokens in multiple batches in the input set.

In [32]:
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=4, stride=4,
    shuffle=False
)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print(f"Inputs:\n{inputs}\n")
print(f"Targets:\n{targets}")

Inputs:
tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Targets:
tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


In [33]:
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=2, stride=2,
    shuffle=False
)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print(f"Inputs:\n{inputs}\n")
print(f"Targets:\n{targets}")

Inputs:
tensor([[   40,   367],
        [ 2885,  1464],
        [ 1807,  3619],
        [  402,   271],
        [10899,  2138],
        [  257,  7026],
        [15632,   438],
        [ 2016,   257]])

Targets:
tensor([[  367,  2885],
        [ 1464,  1807],
        [ 3619,   402],
        [  271, 10899],
        [ 2138,   257],
        [ 7026, 15632],
        [  438,  2016],
        [  257,   922]])


With max_length = 8 and stride = 2, given token_ids will appear in 4 or 5 input batches.

In [34]:
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=8, stride=2,
    shuffle=False
)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print(f"Inputs:\n{inputs}\n")
print(f"Targets:\n{targets}")

Inputs:
tensor([[   40,   367,  2885,  1464,  1807,  3619,   402,   271],
        [ 2885,  1464,  1807,  3619,   402,   271, 10899,  2138],
        [ 1807,  3619,   402,   271, 10899,  2138,   257,  7026],
        [  402,   271, 10899,  2138,   257,  7026, 15632,   438],
        [10899,  2138,   257,  7026, 15632,   438,  2016,   257],
        [  257,  7026, 15632,   438,  2016,   257,   922,  5891],
        [15632,   438,  2016,   257,   922,  5891,  1576,   438],
        [ 2016,   257,   922,  5891,  1576,   438,   568,   340]])

Targets:
tensor([[  367,  2885,  1464,  1807,  3619,   402,   271, 10899],
        [ 1464,  1807,  3619,   402,   271, 10899,  2138,   257],
        [ 3619,   402,   271, 10899,  2138,   257,  7026, 15632],
        [  271, 10899,  2138,   257,  7026, 15632,   438,  2016],
        [ 2138,   257,  7026, 15632,   438,  2016,   257,   922],
        [ 7026, 15632,   438,  2016,   257,   922,  5891,  1576],
        [  438,  2016,   257,   922,  5891,  1576,   438,

### Creating the token embeddings

In [35]:
# Imagine we have a small vocab and want to make embeddings with 3 dimensions
input_ids = torch.tensor([2, 3, 5, 1])
vocab_size = 6
output_dim = 3

torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


In [36]:
print(embedding_layer(torch.tensor([3])))

tensor([[-0.4015,  0.9666, -1.1481]], grad_fn=<EmbeddingBackward0>)


In [37]:
print(embedding_layer(input_ids))

tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)
